In [1]:
import wrds
import pandas as pd
import numpy as np
import datetime
from pandas.tseries.offsets import MonthEnd

In [2]:
from pk_integrity import *
from post_event_nan import *

In [3]:
from compd_fund import *
from compd_aco_pnfnd import *
from crsp_sf import *
from merge_comp_crsp import *
from dff_be import *

In [8]:
def calculate_be(df):
    """
    BE = Share Equity(se) - Prefered Stocks(ps) + Deferred Taxes(dt) - Post retirement Benefit Assets(prba)

    Parameters
    ----------
    df: data frame
        Compustat table with columns ['seq', 'ceq', 'at', 'lt', 'mib', 'pstkrv', 'pstkl', 'pstk', 'txditc', 'txdb', 'itcb', 'prba']

    Definition:
    -----------
    BE is the stockholders book equity, plus balance sheet deferred taxes and investment tax credit (if available),  minus the book value
    of preferred stock. Depending on availability, we use redemption, liquidation, or par value (in that order) for the book value of preferred stock.
    Stockholders equity is the value reported by Moody or COMPUSTAT, if it is available.
    If not, we measure stockholders equity as the book value of common equity plus the par value of preferred stock,
    or the book value of assets minus total liabilities (in that order)".* DFF, JF, 2000, pg 393.
    This is the definition posted on Ken French website:

    """

    required_cols = ['fyear', 'seq', 'ceq', 'at', 'lt', 'mib', 'pstkrv', 'pstkl', 'pstk', 'txditc', 'txdb', 'itcb', 'prba']

    assert set(required_cols).issubset(df.columns), 'Following funda dataitems needed: {}'.format(required_cols)

    df = df[required_cols].copy()

    # Shareholder Equity
    df['se'] = df['seq']

    # Uses Common Equity (ceq) + Preferred Stock (pstk) if SEQ is missing:
    df['se'].fillna((df['ceq'] + df['pstk']), inplace=True)

    # Uses Total Assets (at) - Liabilities (lt) + Minority Interest (mib, if available), if others are missing
    df['se'].fillna((df['at'] - df['lt'] + df['mib'].fillna(0)), inplace=True)

    # Preferred Stock
    # Preferred Stock (Redemption Value)
    df['ps'] = df['pstkrv']
    # Uses Preferred Stock (Liquidating Value (pstkl)) if Preferred Stock (Redemption Value) is missing
    df['ps'].fillna(df['pstkl'], inplace=True)
    # Uses Preferred Stock (Carrying Value (pstk)) if others are missing
    df['ps'].fillna(df['pstk'], inplace=True)

    # Deferred Taxes
    # Uses Deferred Taxes and Investment Tax Credit (txditc)
    df['dt'] = df['txditc']

    # This was Novy-Marx old legacy code. We drop this part to be in accordance with Ken French.
    # Uses Deferred Taxes and Investment Tax Credit(txdb) + Investment Tax Credit (Balance Sheet) (itcb) if txditc is missing

    df['dt'].fillna((df['txdb'].fillna(0) + df['itcb'].fillna(0)), inplace=True)
    # If all measures are missing, set dt to missing
    df.loc[pd.isnull(df['txditc']) & pd.isnull(df['txdb']) & pd.isnull(df['itcb']), 'dt'] = np.nan

    df.loc[df['fyear'] >= 1993, 'dt'] = 0

    # Book Equity
    # Book Equity (BE) = Share Equity (se) - Prefered Stocks (ps) + Deferred Taxes (dt)
    BE = (df['se']  # shareholder equity must be available, otherwise BE is missing
          - df['ps']  # preferred stock must be available, otherwise BE is missing
          + df['dt'].fillna(0)  # add deferred taxes if available
          - df['prba'].fillna(0))  # subtract postretirement benefit assets if available

    return BE


def calculate_op(df):
    """
    Operating Profitability (OP)
    Revenues (SALE (? not sure)) minus cost of goods sold (ITEM 41/COGS), minus selling, general, and administrative expenses (ITEM 189/XSGA),
    minus interest expense (ITEM 15/XINT (? Not Sure)) all divided by book equity.

    Fama, French (2015, JFE, pg.3)
    Ken Frech's website:
    The portfolios for July of year t to June of t+1 include all NYSE, AMEX, and NASDAQ stocks for which we have ME for December of t-1 and June of t,
    (positive) BE for t-1, non-missing revenues data for t-1, and non-missing data for at least one of the following: cost of goods sold, selling,
    general and administrative expenses, or interest expense for t-1.
    """
    required_cols = ['sale', 'cogs', 'xsga', 'xint', 'be']

    assert set(required_cols).issubset(df.columns), 'Following funda dataitems needed: {}'.format(required_cols)

    df = df[required_cols].copy()

    df['cost'] = df[['cogs', 'xsga', 'xint']].sum(axis=1, skipna=True)
    df.loc[df[['cogs', 'xsga', 'xint']].isnull().all(axis=1), 'cost'] = np.nan

    df['op'] = df['sale'] - df['cost']
    df.loc[(df.be > 0), 'opbe'] = df['op'] / df['be']

    return df['op'], df['opbe']


def calculate_inv(df):
    """
    Investment (INV)
    The change in total assets from the fiscal year ending in year t-2 to the fiscal year ending in t-1, divided by t-2 total assets.
    Fama, French (2015, JFE, pg.3)

    Notes:
    ------
    Ken Frech's website:
    The portfolios for July of year t to June of t+1 include all NYSE, AMEX, and NASDAQ stocks for which we have market equity data for June of t and total assets
    data for t-2 and t-1.
    """

    required_cols = ['datadate', 'gvkey', 'fyear', 'at']

    assert set(required_cols).issubset(df.columns), 'Following funda dataitems needed: {}'.format(required_cols)

    df = df[required_cols].copy()

    if any(df.fyear.isnull() & df['at'].notnull()):
        warnings.warn('''Missing fyear with valid at value. Row was arbitrarily deleted. ''')

    # Notice that [gvkey, fyear] is not a primary key for compustat.
    # There are some cases in which there are 2 datadate for the same fyear.
    # In many cases there is one of them have missing.  In this case we keep the entry that is not missing.
    df = df[df.fyear.notnull()]
    df = df[df['at'] > 0]

    pk_integrity(df, ['gvkey', 'fyear'])

    df.sort_values(['gvkey', 'fyear'], inplace=True)
    df['lag_at'] = df.groupby('gvkey').at.shift(1)
    df['inv'] = (df['at'] - df['lag_at']) / df['lag_at']

    # Take care if there are years missing
    df['fdiff'] = df.groupby('gvkey').fyear.diff()
    df.loc[df.fdiff > 1, 'inv'] = np.nan

    # df[df['gvkey'] == '018073']

    return df['inv']

# %% Main Function


def main(save_out=True):

    print("Stock annual calculation.")
    # %% Set Up
    db = wrds.Connection(wrds_username='wcwenjin')  # make sure to configure wrds connector before hand.
    DATAPATH = "/Users/wenjincao/Google Drive/Columbia MSFE/Big Data/Homework/hm_ii/" # where to save output?

    start_time = time.time()

    # %% Import Data

    varlist = ['conm', 'fyear', 'fyr', 'at', 'capx', 'ceq', 'cogs', 'dlc', 'ib', 'icapt', 'itcb', 'lt', 'mib',
               'naicsh', 'pstk', 'pstkl', 'pstkrv',  'sale', 'seq', 'sich', 'sstk', 'txdb', 'txdi', 'txditc',
               'xint', 'xsga']
    start_date = '1950-01-01'  # '2017-01-01'#
    end_date = datetime.date.today().strftime("%Y-%m-%d")
    freq = 'annual'

    # Download firms fundamentals from Compustat.
    comp_data = compd_fund(varlist=varlist, start_date=start_date, end_date=end_date, freq='annual', db=db)

    del varlist

    # Download pension data
    varlist_aco = ['prba']
    pension_data = compd_aco_pnfnd(varlist=varlist_aco, start_date=start_date, end_date=end_date, freq=freq, db=db)

    del varlist_aco

    # Download Davis, Fama, French BE Data
    dff = dff_be()

    # CRSP ME Data
    varlist_crsp = varlist = ['exchcd', 'naics', 'permco', 'prc', 'shrcd', 'shrout', 'siccd', 'ticker']
    start_date = '1925-01-01'  # '2017-01-01' #
    end_date = datetime.date.today().strftime("%Y-%m-%d")
    freq = 'monthly'  # 'daily'
    crspm = crsp_sf(varlist_crsp,
                    start_date,
                    end_date,
                    freq=freq,
                    db=db)

    del varlist_crsp

    # Merge
    comp = pd.merge(comp_data, pension_data, on=['gvkey', 'datadate'], how='left')

    del (comp_data, pension_data)

    # %% Add variables

    # Add BE
    comp['be'] = calculate_be(comp)

    # Add OP: naming here is to be consistent with risk and returns projects.
    comp['op'], comp['opbe'] = calculate_op(comp)

    # Add INV
    comp['inv_gvkey'] = calculate_inv(comp)
    # comp.loc[comp.gvkey == '006557', ['gvkey', 'datadate', 'fyear', 'inv']]
    # comp.loc[comp.gvkey == '001414', ['gvkey','datadate','fyear','inv','at']]

    # There are 2 entries in which fyear is missing. All variables are null in these cases.
    # comp.loc[comp['fyear'].isnull(), ['gvkey', 'cusip', 'permno', 'fyear', 'at', 'be']]
    comp.dropna(subset=['fyear'], inplace=True)

    # Merged data set has [permno, datadate] as primary key. We need ['permno', 'fyear'] as primary key.
    # Six duplicated cases in which we choose the latest observation.
    # comp[comp[['permno', 'fyear']].duplicated(keep=False)][['permno', 'gvkey', 'datadate', 'fyear', 'at']]
    # comp[comp['permno']==22074][['permno', 'gvkey', 'datadate', 'fyear', 'at']]

    # These are cases where the gvkey changes (probably merger) and the fiscal-year-end (fyr) also changes
    # by keeping last, we put the merger investment into the year it happens
    # That seems reasonable
    comp = comp[~comp.duplicated(subset=['gvkey', 'fyear'], keep='last')]

    # Add rankyear
    comp['rankyear'] = comp['fyear'] + 1

    # %% Add CRSP Variables

    # Link CRSP/Compustat table
    lcomp = merge_compustat_crsp(comp, db=db)
    # Link CRSP/Compustat table
    lcomp = merge_compustat_crsp(comp, db=db)
    print("CRSP and Compsuat merge created %d (fyear, permno) duplicates." %lcomp[lcomp.duplicated(subset=['permno', 'fyear'])].shape[0])
    print("Keeping only the last available datadate per PERMNO.")

    lcomp.sort_values(by=['permno', 'fyear', 'datadate'])
    lcomp = lcomp[~lcomp.duplicated(subset=['permno', 'fyear'], keep='last')]

    #  Calculate INV by PERMCO
    lcomp.sort_values(['permno', 'fyear'], inplace=True)
    lcomp['lag_at'] = lcomp.groupby('permno').at.shift(1)
    lcomp['inv'] = (lcomp['at'] - lcomp['lag_at']) / lcomp['lag_at']
    # Take care if there are years missing
    lcomp['fdiff'] = lcomp.groupby('permno').fyear.diff()
    lcomp.loc[lcomp.fdiff > 1, 'inv'] = np.nan
    lcomp.loc[(lcomp['at'] <= 0), 'inv'] = np.nan
    lcomp.loc[(lcomp['lag_at'] <= 0), 'inv'] = np.nan
    lcomp.drop(columns=['lag_at', 'fdiff'], inplace=True)

    # Add Davis data
    dff.rename(columns={'be': 'be_dff'}, inplace=True)
    # Add PERMCO to DFF data: Important for ME sum later
    pp_key = crspm[['permno', 'permco']].drop_duplicates()
    # max(pp_key.groupby(['permno']).permco.count())
    dff = pd.merge(dff, pp_key, on=['permno'], how='left')

    print('There are %d PERMNOs without a valid PERMCO: not present in the crspm table.' % dff[dff.permco.isnull()].permno.unique())

    lcomp = pd.merge(lcomp, dff, on=['permno', 'permco', 'rankyear'], how='outer')
    lcomp.be.fillna(lcomp['be_dff'], inplace=True)
    lcomp.drop(columns=['be_dff'], inplace=True)
    lcomp.fyear.fillna(lcomp['rankyear'] - 1, inplace=True)

    print('Number of not valid PERMCOs lcomp: %d' % round(lcomp.permco.isnull().sum() / lcomp.shape[0], 4))

    del dff, pp_key
    ## Notice that, since int does not support null, outer merge changes dtype of permco

    lcomp.sort_values(['permno', 'rankyear'], inplace=True)

    # ME, SICCD, TICKER, exchcd and shrcrd from June
    crspm['me'] = crspm.prc.abs()*crspm.shrout

    crspjune = crspm.loc[crspm.date.dt.month == 6, ['permno', 'permco', 'date', 'me', 'exchcd', 'shrcd', 'ticker', 'siccd']]
    crspjune['rankyear'] = crspjune.date.dt.year
    crspjune.drop('date', axis=1, inplace=True)
    stock_annual = lcomp.merge(crspjune, how='outer', on=['permno', 'permco', 'rankyear'])
    stock_annual.sort_values(['permno', 'rankyear'], inplace=True)

    # For summing size over issues of the same firm:
    # we rely on gvkey first
    # and if there is no gvkey (e.g. before lcompustat sample period) we use PERMCO
    stock_annual['gvkey_permco'] = stock_annual.gvkey.fillna(stock_annual['permco'])
    stock_annual['mesum_june'] = stock_annual.groupby(['fyear', 'gvkey_permco'])['me'].transform(np.sum, min_count=1)
    stock_annual.rename(columns={'me': 'mejune'}, inplace=True)
    del crspjune

    # Calculate ME december
    crspdec = crspm.loc[crspm.date.dt.month == 12, ['permno', 'date', 'me']]
    crspdec['fyear'] = crspdec.date.dt.year
    crspdec.drop('date', axis=1, inplace=True)
    crspdec.rename(columns={'me': 'medec'}, inplace=True)
    stock_annual = pd.merge(stock_annual, crspdec, how='left', on=['permno', 'fyear'])
    stock_annual['mesum_dec'] = stock_annual.groupby(['fyear', 'gvkey_permco'])['medec'].transform(np.sum, min_count=1)
    del crspdec, lcomp

    # Calculate book-to-market
    # In accordance with FF1993 and DFF2000 the BEME used to form portfolios in June of year t,
    # is BE for the fiscal year ending in t-1, divided by ME at December of t-1. ME for December
    stock_annual.loc[(stock_annual.be > 0), 'beme'] = stock_annual['be'] / stock_annual['mesum_dec']
    stock_annual.loc[(stock_annual.mesum_dec == 0) | (stock_annual.mesum_dec.isnull()), 'beme'] = np.nan

    # Calculate again variables that depend on be values. Need to consider the DFF data OP
    stock_annual.loc[(stock_annual.be > 0), 'opbe'] = stock_annual['op'] / stock_annual['be']

    # Add CRSP SIC when missing
    stock_annual.sich.fillna(stock_annual['siccd'], inplace=True)

    # Back fill SIC code
    stock_annual['sich_filled'] = stock_annual.sich.copy()
    stock_annual['sich_filled'] = stock_annual.groupby('permno').sich_filled.fillna(method='bfill')
    stock_annual['sich_filled'] = stock_annual.groupby('permno').sich_filled.fillna(method='ffill')

    print('Number of entries with valis sich:')
    print(round(pd.isnull(stock_annual.sich).sum() / stock_annual.shape[0], 4))
    print('Number of entries with valis sich_filled:')
    print(round(pd.isnull(stock_annual.sich_filled).sum() / stock_annual.shape[0], 4))

    pk_integrity(stock_annual, ['permno', 'rankyear'])
    stock_annual.sort_values(['permno', 'rankyear'], inplace=True)

    stock_annual.drop(columns=['gvkey_permco'], inplace=True)

    print("Time to create stock_annual: %s seconds" % str(time.time() - start_time))
    
    if save_out:
        stock_annual.to_pickle(DATAPATH + 'stock_annual.pkl')
        print("Successfully saved stock_annual.")

    return stock_annual

# %% Main
if __name__ == '__main__':
    main()


Stock annual calculation.
Enter your WRDS username [wenjincao]:wcwenjin
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done
Compustat data was successfully downloaded in 15.011133909225464 seconds.
Pension data was successfully downloaded in 0.6643712520599365 seconds.
CRSP data was successfully downloaded in 99.27138090133667 seconds.
CRSP and Compsuat merge created 6 (fyear, permno) duplicates.
Keeping only the last available datadate per PERMNO.
There are 81664 PERMNOs without a valid PERMCO: not present in the crspm table.
Number of not valid PERMCOs lcomp: 0
Number of entries with valis sich:
0.0452
Number of entries with valis sich_filled:
0.003
Time to create stock_annual: 178.13454604148865 seconds
Successfully saved stock_annual.


In [9]:
# %% Main Function

def main(save_out=True):

    # %% Set Up
    db = wrds.Connection(wrds_username='wcwenjin')  # make sure to configure wrds connector before hand.
    DATAPATH = "/Users/wenjincao/Google Drive/Columbia MSFE/Big Data/Homework/hm_ii/" # where to save output?

    start_time = time.time()

    # %% Download CRSP data
    varlist = ['dlret', 'dlretx', 'exchcd', 'naics', 'permco', 'prc', 'ret', 'shrcd', 'shrout', 'siccd', 'ticker']

    start_date = '2000-01-01' # '1970-01-01'
    end_date = datetime.date.today().strftime("%Y-%m-%d")
    freq = 'monthly'  # 'daily'
    permno_list = None  # [10001, 14593, 10107] #
    shrcd_list = None  # [10, 11] #
    exchcd_list = None  # [1, 2, 3] #
    crspm = crsp_sf(varlist,
                    start_date,
                    end_date,
                    freq=freq,
                    permno_list=permno_list,
                    shrcd_list=shrcd_list,
                    exchcd_list=exchcd_list,
                    db=db)

    # %% Create variables

    # Returns adjusted for delisting
    crspm['retadj'] = ((1 + crspm['ret'].fillna(0)) * (1 + crspm['dlret'].fillna(0)) - 1)
    crspm.loc[crspm[['ret', 'dlret']].isnull().all(axis=1), 'retadj'] = np.nan

    # Create Market Equity (ME)
    # SHROUT is the number of publicly held shares, recorded in thousands. ME will be reported in 1,000,000 ($10^6$).
    # If the stock is delisted, we set ME to NaN.
    # Also, some companies have multiple shareclasses (=PERMNOs).
    # To get the company ME, we need to calculate the sum of ME over all shareclasses for one company (=PERMCO).
    # This is used for sorting, but not for weights.
    crspm['me'] = abs(crspm['prc']) * (crspm['shrout'] / 1000)

    # Create MEsum
    crspm['mesum_permco'] = crspm.groupby(['date', 'permco']).me.transform(np.sum, min_count=1)

    # Adjust for delisting
    crspm.loc[crspm.dlret.notnull(), 'me'] = np.nan
    crspm.loc[crspm.dlret.notnull(), 'mesum'] = np.nan

    # Resample data
    # CRSP data has skipping months.
    # Create line to missing  months to facilitate the calculation of lag/past returns

    # -- Check
    crspm.shape
    crspm['days_diff'] = crspm.groupby('permno').date.diff()
    crspm.days_diff.max()

    # -- Resample
    crspm['cpermno'] = 'c' + crspm['permno'].astype(str)
    dcast_crspm = pd.pivot_table(crspm, index='date', columns='cpermno', values='ret').reset_index()

    complete_crsp = pd.wide_to_long(dcast_crspm, stubnames='c', i="date", j='permno')
    complete_crsp.reset_index(inplace=True)

    # -- Fist Date
    fdate = crspm.groupby('permno').date.min().reset_index()
    fdate.rename(columns={'date': 'fdate'}, inplace=True)

    # -- Last Date
    edate = crspm.groupby('permno').date.max().reset_index()
    edate.rename(columns={'date': 'edate'}, inplace=True)

    # -- Date range: select dates within the date range
    drange = pd.merge(fdate, edate)
    complete_crsp = pd.merge(complete_crsp[['date', 'permno']], drange, on=['permno'])
    complete_crsp = complete_crsp[complete_crsp.date >= complete_crsp.fdate]
    complete_crsp = complete_crsp[complete_crsp.date <= complete_crsp.edate]
    complete_crsp.shape

    crspm = pd.merge(complete_crsp[['date', 'permno', 'edate']],
                     crspm,
                     on=['date', 'permno'],
                     how='left')

    crspm.drop(columns='cpermno', inplace=True)

    crspm['days_diff'] = crspm.groupby('permno').date.diff()
    crspm.days_diff.max()
    crspm.days_diff.min()

    # Create MElag
    crspm['lag_me'] = crspm.groupby('permno').me.shift(1)
    crspm['lag_dlret'] = crspm.groupby('permno').dlret.shift(1)

    crspm.sort_values(['permno', 'date'], inplace=True)
    crspm.set_index(['date', 'permno'], inplace=True)
    crspm['melag'] = calculate_melag(crspm)

    # Delete rows that were not in the original data set
    crspm.drop(columns=[x for x in crspm.columns if 'lag_' in x], inplace=True)
    crspm.drop(columns=['edate'], inplace=True)

    # TODO: Calculate past 11, 1 returns

    print("Time to create CRSP monthly: %s seconds" % str(time.time() - start_time))

    # Rankyear
    # Rankyear is the year where we ranked the stock, e.g., for the return of a stock in January 2001,
    # rankyear is 2000, because we ranked it in June 2000
    crspm.reset_index(inplace=True)
    crspm['rankyear'] = crspm.date.dt.year
    crspm.loc[crspm.date.dt.month <= 6, 'rankyear'] = crspm.loc[crspm.date.dt.month <= 6, 'rankyear'] - 1

    if save_out:
        crspm.to_pickle(DATAPATH+'stock_monthly.pkl')
        print("Successfully saved stock_monthly.")
    return crspm


# %% Main
if __name__ == '__main__':
    main()


Enter your WRDS username [wenjincao]:wcwenjin
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done
CRSP data was successfully downloaded in 49.87037014961243 seconds.
Time to create CRSP monthly: 95.34511208534241 seconds
Successfully saved stock_monthly.


In [13]:
stock_annual = pd.read_pickle('/Users/wenjincao/Google Drive/Columbia MSFE/Big Data/Homework/hm_ii/stock_annual.pkl')
stock_annual

,gvkey,datadate,conm,fyear,fyr,at,capx,ceq,cogs,dlc,...,mejune,exchcd,shrcd,ticker,siccd,mesum_june,medec,mesum_dec,beme,sich_filled
0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.173459e+04,3.0,10.0,OMFGA,3990.0,NaN,NaN,NaN,NaN,3990.0
1,013007,1986-10-31,OPTIMUM MANUFACTURING -CL A,1986.0,10.0,2.115,0.240,0.418,0.511,0.968,...,NaN,3.0,10.0,OMFGA,3990.0,NaN,1.981547e+03,1.981547e+03,0.000211,3990.0
2,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.033125e+03,3.0,11.0,GFGC,4920.0,NaN,NaN,NaN,NaN,4920.0
3,012994,1986-06-30,GAS NATURAL INC,1986.0,6.0,12.242,0.551,5.432,19.565,0.343,...,5.822125e+03,3.0,11.0,GFGC,4920.0,5.822125e+03,6.937000e+03,6.937000e+03,0.001014,4920.0
4,012994,1987-06-30,GAS NATURAL INC,1987.0,6.0,11.771,0.513,5.369,15.538,0.377,...,6.200000e+03,3.0,11.0,GFGC,4920.0,6.200000e+03,5.828000e+03,5.828000e+03,0.001208,4924.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395582,184996,2015-12-31,TESLA INC,2015.0,12.0,8092.460,1634.850,1088.944,2699.932,633.166,...,3.142062e+07,3.0,11.0,TSLA,9999.0,3.142062e+07,3.154331e+07,3.154331e+07,0.000036,3711.0
395583,184996,2016-12-31,TESLA INC,2016.0,12.0,22664.076,1440.471,4752.911,4453.776,1202.178,...,6.033933e+07,3.0,11.0,TSLA,9999.0,6.033933e+07,3.452397e+07,3.452397e+07,0.000138,3711.0
395584,184996,2017-12-31,TESLA INC,2017.0,12.0,28655.372,4081.354,4237.242,7900.261,963.862,...,5.847846e+07,3.0,11.0,TSLA,9999.0,5.847846e+07,5.255495e+07,5.255495e+07,0.000081,3711.0
395585,184996,2018-12-31,TESLA INC,2018.0,12.0,29739.614,2319.516,4923.243,15531.461,2629.460,...,4.002571e+07,3.0,11.0,TSLA,9999.0,4.002571e+07,5.744194e+07,5.744194e+07,0.000086,3711.0


In [12]:
stock_monthly = pd.read_pickle('/Users/wenjincao/Google Drive/Columbia MSFE/Big Data/Homework/hm_ii/stock_monthly.pkl')
stock_monthly

,date,permno,dlret,dlretx,exchcd,naics,permco,prc,ret,shrcd,shrout,siccd,ticker,retadj,me,mesum_permco,mesum,days_diff,melag,rankyear
0,2000-01-31,10001,NaN,NaN,3.0,None,7953.0,8.125000,-0.044118,11.0,2450.0,4920.0,EWST,-0.044118,19.906250,19.906250,NaN,NaT,NaN,1999
1,2000-02-29,10001,NaN,NaN,3.0,None,7953.0,8.250000,0.015385,11.0,2450.0,4920.0,EWST,0.015385,20.212500,20.212500,NaN,29 days,NaN,1999
2,2000-03-31,10001,NaN,NaN,3.0,None,7953.0,-8.000000,-0.015758,11.0,2464.0,4920.0,EWST,-0.015758,19.712000,19.712000,NaN,31 days,NaN,1999
3,2000-04-28,10001,NaN,NaN,3.0,None,7953.0,-8.093750,0.011719,11.0,2464.0,4920.0,EWST,0.011719,19.943000,19.943000,NaN,28 days,NaN,1999
4,2000-05-31,10001,NaN,NaN,3.0,None,7953.0,-7.906250,-0.023166,11.0,2464.0,4920.0,EWST,-0.023166,19.481000,19.481000,NaN,33 days,NaN,1999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1735707,2019-08-30,93436,NaN,NaN,3.0,336111,53453.0,225.610001,-0.066222,11.0,179127.0,9999.0,TSLA,-0.066222,40412.842579,40412.842579,NaN,30 days,NaN,2019
1735708,2019-09-30,93436,NaN,NaN,3.0,336111,53453.0,240.869995,0.067639,11.0,180000.0,9999.0,TSLA,0.067639,43356.599121,43356.599121,NaN,31 days,NaN,2019
1735709,2019-10-31,93436,NaN,NaN,3.0,336111,53453.0,314.920013,0.307427,11.0,180245.0,9999.0,TSLA,0.307427,56762.757820,56762.757820,NaN,31 days,NaN,2019
1735710,2019-11-29,93436,NaN,NaN,3.0,336111,53453.0,329.940002,0.047695,11.0,180245.0,9999.0,TSLA,0.047695,59470.035740,59470.035740,NaN,29 days,NaN,2019


In [33]:
def calculate_cumulative_returns(mdata, tt, min_periods): # TODO: to be completed
    """
    Calculate past returns for momentum strategy

    Parameters:
    ------------
    mdata: data frame
        crsp monthly data with cols permno, date as index.
    tt: int
        number of periods to cumulate retuns
    min_periods: int
        minimum number of periods. Default tt/2
    """
    start_time = time.time()

    required_cols = ['retadj','ret']

    assert set(required_cols).issubset(mdata.columns), "Required columns: {}.".format(', '.join(required_cols))

    df = mdata[required_cols].copy()
    df['retadj'] = df['retadj']+1
    df['ret'] = df['retadj'].notnull()

    df.reset_index(level=0, inplace=True)
    

    # Cumulative Return (adjusted) in 11 month
    cret = (df[['retadj']]).rolling(window=tt+1, min_periods = min_periods).\
                        apply(lambda x : np.prod(x), raw=True) /df[['retadj']]
    

    print("Time to calculate %d months past returns: %s seconds" % (tt, str(round(time.time() - start_time, 2))))

    return cret


def calculate_melag(mdata):
    """
     Parameters:
    ------------
    mdata: data frame
        crsp monthly data with cols permno, date as index and lag_me column

    Notes:
    ------
    If ME is missing, we do not exclude stock, but rather keep it in with last non-missing MElag.
    The stock will be excluded if:
    (i) Delisted;
    (ii) Have a missing ME in the moment of portfolio construction.

    This is different than Ken's method

    EXAMPLE:
    --------
    there seem to be 12 stocks with missing PRC and thus missing ME in July 1926.
    Thus, our number of firms drops from 428 to 416.
    Fama and French report 427 in both July and August, so they also do not seem to exclude these
    rather they probably use the previous MElag for weight and must assume some return in the following month.

    The whole paragraph from the note on Ken French's website:
    ----------------------------------------------------------
    "In May 2015, we revised the method for computing daily portfolio returns
    to match more closely the method for computing monthly portfolio returns.
    Daily files produced before May 2015 drop stocks from a portfolio
    (i) the next time the portfolio is reconstituted, at the end of June, regardless of the CRSP delist date or
    (ii) during any period in which they are missing prices for more than 10 consecutive trading days.
    Daily files produced after May 2015 drop stocks from a portfolio
    (i) immediately after their CRSP delist date or
    (ii) during any period in which they are missing prices for more than 200 consecutive trading days. "
    """

    required_cols = ['edate', 'lag_me', 'lag_dlret']

    set(required_cols).issubset(mdata.columns), "Required columns: {}.".format(', '.join(required_cols))

    df = mdata[required_cols].copy()
    df['melag'] = df.groupby('permno').lag_me.fillna(method='pad')
    df.reset_index(inplace=True)

    # Fill na after delisting
    df = post_event_nan(df=df,
                        event=df.lag_dlret.notnull(),
                        vars=['melag'],
                        id_vars=['permno', 'edate'])

    df.set_index(['permno', 'date'], inplace=True)

    return df[['melag']]


In [35]:
stock_monthly['rolling_cret'] = calculate_cumulative_returns(stock_monthly, 11, 6)

Time to calculate 11 months past returns: 6.31 seconds


In [36]:
stock_monthly

,date,permno,dlret,dlretx,exchcd,naics,permco,prc,ret,shrcd,...,siccd,ticker,retadj,me,mesum_permco,mesum,days_diff,melag,rankyear,rolling_cret
0,2000-01-31,10001,NaN,NaN,3.0,None,7953.0,8.125000,-0.044118,11.0,...,4920.0,EWST,-0.044118,19.906250,19.906250,NaN,NaT,NaN,1999,NaN
1,2000-02-29,10001,NaN,NaN,3.0,None,7953.0,8.250000,0.015385,11.0,...,4920.0,EWST,0.015385,20.212500,20.212500,NaN,29 days,NaN,1999,NaN
2,2000-03-31,10001,NaN,NaN,3.0,None,7953.0,-8.000000,-0.015758,11.0,...,4920.0,EWST,-0.015758,19.712000,19.712000,NaN,31 days,NaN,1999,NaN
3,2000-04-28,10001,NaN,NaN,3.0,None,7953.0,-8.093750,0.011719,11.0,...,4920.0,EWST,0.011719,19.943000,19.943000,NaN,28 days,NaN,1999,NaN
4,2000-05-31,10001,NaN,NaN,3.0,None,7953.0,-7.906250,-0.023166,11.0,...,4920.0,EWST,-0.023166,19.481000,19.481000,NaN,33 days,NaN,1999,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1735707,2019-08-30,93436,NaN,NaN,3.0,336111,53453.0,225.610001,-0.066222,11.0,...,9999.0,TSLA,-0.066222,40412.842579,40412.842579,NaN,30 days,NaN,2019,0.800935
1735708,2019-09-30,93436,NaN,NaN,3.0,336111,53453.0,240.869995,0.067639,11.0,...,9999.0,TSLA,0.067639,43356.599121,43356.599121,NaN,31 days,NaN,2019,0.852098
1735709,2019-10-31,93436,NaN,NaN,3.0,336111,53453.0,314.920013,0.307427,11.0,...,9999.0,TSLA,0.307427,56762.757820,56762.757820,NaN,31 days,NaN,2019,0.714070
1735710,2019-11-29,93436,NaN,NaN,3.0,336111,53453.0,329.940002,0.047695,11.0,...,9999.0,TSLA,0.047695,59470.035740,59470.035740,NaN,29 days,NaN,2019,0.898539


In [39]:
num_missing = stock_monthly['rolling_cret'].isna().sum()
num_missing

166503

In [48]:
import sys

sys.path.append('/Users/wenjincao/Google Drive/Columbia MSFE/Big Data/Homework/liramota-fire_pytools-800686b4e28b/utils')
sys.path

['/Users/wenjincao/Google Drive/Columbia MSFE/Big Data/Homework/liramota-fire_pytools-800686b4e28b/utils',
 '/Users/wenjincao/opt/anaconda3/lib/python37.zip',
 '/Users/wenjincao/opt/anaconda3/lib/python3.7',
 '/Users/wenjincao/opt/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/Users/wenjincao/.local/lib/python3.7/site-packages',
 '/Users/wenjincao/opt/anaconda3/lib/python3.7/site-packages',
 '/Users/wenjincao/opt/anaconda3/lib/python3.7/site-packages/aeosa',
 '/Users/wenjincao/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/wenjincao/.ipython',
 '/Users/wenjincao/Google Drive/Columbia MSFE/Big Data/Homework/liramota-fire_pytools-800686b4e28b',
 '/Users/wenjincao/Google Drive/Columbia MSFE/Big Data/Homework/liramota-fire_pytools-800686b4e28b',
 '/Users/wenjincao/Google Drive/Columbia MSFE/Big Data/Homework/liramota-fire_pytools-800686b4e28b/',
 '/Users/wenjincao/Google Drive/Columbia MSFE/Big Data/Homework/liramota-fire_pytools-800686b4e28b/utils']

In [50]:
from monthly_date import *

from find_breakpoints import find_breakpoints
from sort_portfolios import sort_portfolios

In [51]:
desired_width = 10
pd.set_option('display.width', desired_width)
idx = pd.IndexSlice

# Set names
adata = stock_annual.copy()
adata.rename(columns={'mesum_june': 'me', 'inv_gvkey': 'inv'}, inplace=True) #inv_permco

# %% Create Filters
# shrcd must be (10,11)
# ---------------------
print('Data deleted due to shrcd: %f' % np.round((1-adata.shrcd.isin([10, 11]).mean())*100, 2))
sort_data = adata[adata.shrcd.isin([10, 11])].copy()

# exchcd must be (1, 2, 3)
# ------------------------
print('Data deleted due to exchcd: %f' % np.round((1-sort_data.exchcd.isin([1, 2, 3]).mean())*100, 2))
sort_data = sort_data[sort_data.exchcd.isin([1, 2, 3])]

Data deleted due to shrcd: 23.320000
Data deleted due to exchcd: 1.560000


In [52]:
sort_data

,gvkey,datadate,conm,fyear,fyr,at,capx,ceq,cogs,dlc,...,mejune,exchcd,shrcd,ticker,siccd,me,medec,mesum_dec,beme,sich_filled
0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.173459e+04,3.0,10.0,OMFGA,3990.0,NaN,NaN,NaN,NaN,3990.0
1,013007,1986-10-31,OPTIMUM MANUFACTURING -CL A,1986.0,10.0,2.115,0.240,0.418,0.511,0.968,...,NaN,3.0,10.0,OMFGA,3990.0,NaN,1.981547e+03,1.981547e+03,0.000211,3990.0
2,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.033125e+03,3.0,11.0,GFGC,4920.0,NaN,NaN,NaN,NaN,4920.0
3,012994,1986-06-30,GAS NATURAL INC,1986.0,6.0,12.242,0.551,5.432,19.565,0.343,...,5.822125e+03,3.0,11.0,GFGC,4920.0,5.822125e+03,6.937000e+03,6.937000e+03,0.001014,4920.0
4,012994,1987-06-30,GAS NATURAL INC,1987.0,6.0,11.771,0.513,5.369,15.538,0.377,...,6.200000e+03,3.0,11.0,GFGC,4920.0,6.200000e+03,5.828000e+03,5.828000e+03,0.001208,4924.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395581,184996,2014-12-31,TESLA INC,2014.0,12.0,5849.251,969.885,911.710,2084.754,611.098,...,3.409612e+07,3.0,11.0,TSLA,9999.0,3.409612e+07,2.795427e+07,2.795427e+07,0.000035,3711.0
395582,184996,2015-12-31,TESLA INC,2015.0,12.0,8092.460,1634.850,1088.944,2699.932,633.166,...,3.142062e+07,3.0,11.0,TSLA,9999.0,3.142062e+07,3.154331e+07,3.154331e+07,0.000036,3711.0
395583,184996,2016-12-31,TESLA INC,2016.0,12.0,22664.076,1440.471,4752.911,4453.776,1202.178,...,6.033933e+07,3.0,11.0,TSLA,9999.0,6.033933e+07,3.452397e+07,3.452397e+07,0.000138,3711.0
395584,184996,2017-12-31,TESLA INC,2017.0,12.0,28655.372,4081.354,4237.242,7900.261,963.862,...,5.847846e+07,3.0,11.0,TSLA,9999.0,5.847846e+07,5.255495e+07,5.255495e+07,0.000081,3711.0


In [56]:
# %% Portfolio Sorts
## ME X BEME
# notice that the way we defined beme or beme is null if be<=0
sample_filters = ((sort_data.me > 0) &
                  (sort_data.mesum_dec > 0) &
                  (sort_data.beme.notnull()))

beme_sorts = sort_portfolios(data=sort_data[sample_filters],
                             quantiles={'me': [0.5], 'beme': [0.3, 0.7]},
                             id_variables=['rankyear', 'permno', 'exchcd'],
                             exch_cd=[1]
                             )


IndexError: index 105 is out of bounds for size 94

In [ ]:

# TODO: ME X OP

# TODO: ME X INV

# TODO: ME X ret_11_1 (sorts at each month - id_variables=['date', 'permno', 'exchcd'])


